In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import requests
from bs4 import BeautifulSoup
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from newspaper import Article

c:\Users\zfkha\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
finbert = BertForSequenceClassification.from_pretrained('ProsusAI/finbert',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

c:\Users\zfkha\miniconda3\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp("Stock values are dropping at a rapid rate")

In [6]:
url = "https://www.ft.com/content/7979b1a3-ded2-4f27-a563-101920bdb379"
article = Article(url)
article.download()
article.parse()
article.text

'Print this page\n\nTech giant Amazon has struck a deal with the UK’s largest publisher Reach over obtaining customer data to target online advertising, as the media industry scrambles to respond to Google’s move to axe “cookies”.\n\nIn one of the first such agreements in Europe, Amazon and Reach unveiled a partnership on Monday designed to compensate for the loss of “third party” cookies that help gather information about users by tracking their activity across websites to help target advertising.\n\nGoogle said this month that it had started to remove cookies on its Chrome browser, following a similar move by Apple to block them over Safari, aiming to switch off all third-party cookies by the end of the year.\n\nReach said it will partner with Amazon on sharing “contextual” first-party data, for example allowing advertisers to know what articles people are looking at, with the US tech group using the information to sell more targeted advertising on the UK publisher’s sites.\n\nThe co

In [12]:
def summarise(text, percentage=0.2):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in STOP_WORDS and word.text.lower() not in punctuation:
            word_frequencies[word.text.lower()] = word_frequencies.get(word.text.lower(), 0) + 1

    max_frequency = max(word_frequencies.values())
    normalized_frequencies = {word: freq / max_frequency for word, freq in word_frequencies.items()}

    sentence_scores = {}
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() in normalized_frequencies:
                sentence_scores[sent] = sentence_scores.get(sent, 0) + normalized_frequencies[word.text.lower()]

    summary_length = int(len(list(doc.sents)) * percentage)
    summary_sentences = nlargest(summary_length, sentence_scores, key=sentence_scores.get)
    summary = ' '.join([sent.text for sent in summary_sentences])

    return summary

In [18]:
summary = summarise(article.text)
result = round(nlp(summary)[0]['score'],2)
result

0.82